# LNDL Prompt - LLM Instruction Generation

LNDL (Language Natural Declaration Language) provides system prompts for guiding LLMs to generate structured outputs while maintaining natural thinking patterns.

**Core Features:**
- **Variable Declaration**: `<lvar Model.field alias>value</lvar>` for explicit field mapping
- **Action Execution**: `<lact>function_call(args)</lact>` for tool/function calls
- **Natural Thinking**: Intermix prose, reasoning, and structured declarations
- **Explicit Output**: `OUT{field:[vars]}` syntax for final structured results
- **Lazy Execution**: Actions only execute if referenced in OUT{} block
- **Type Safety**: Model.field namespacing prevents ambiguity

In [1]:
from lionherd_core.lndl.prompt import get_lndl_system_prompt

## 1. Basic Usage - Getting the System Prompt

The module provides a simple function to retrieve the LNDL system prompt for LLM guidance.

In [2]:
# Get the system prompt
prompt = get_lndl_system_prompt()

print(f"Prompt length: {len(prompt)} characters")
print(f"Prompt lines: {len(prompt.splitlines())} lines")
print(f"\nFirst 200 characters:\n{prompt[:200]}...")

Prompt length: 4673 characters
Prompt lines: 124 lines

First 200 characters:
LNDL - Structured Output with Natural Thinking

SYNTAX

Variables:
<lvar Model.field alias>value</lvar>

- Model.field: Explicit mapping (Report.title, Reason.confidence)
- alias: Short name for OUT{}...


## 2. LNDL Variable Syntax - Explicit Field Mapping

LNDL variables use `<lvar Model.field alias>value</lvar>` syntax for explicit field mapping.

In [3]:
# Example LNDL variable declarations
example_vars = """
Let me analyze the report structure:

<lvar Report.title t>AI Safety Analysis</lvar>
<lvar Report.summary s>This report examines current AI safety practices...</lvar>
<lvar Report.author a>Research Team</lvar>

I could also phrase it differently:
<lvar Report.summary s2>An investigation into AI safety protocols...</lvar>

```lndl
OUT{report:[t, s, a]}
```
"""

print("LNDL Variable Example:")
print(example_vars)
print("\nKey Points:")
print("- Model.field provides explicit mapping (Report.title, Report.summary)")
print("- Alias (t, s, a) for short reference in OUT{}")
print("- Multiple versions allowed (s, s2), select final in OUT{}")
print("- Natural prose mixed with declarations")

LNDL Variable Example:

Let me analyze the report structure:

<lvar Report.title t>AI Safety Analysis</lvar>
<lvar Report.summary s>This report examines current AI safety practices...</lvar>
<lvar Report.author a>Research Team</lvar>

I could also phrase it differently:
<lvar Report.summary s2>An investigation into AI safety protocols...</lvar>

```lndl
OUT{report:[t, s, a]}
```


Key Points:
- Model.field provides explicit mapping (Report.title, Report.summary)
- Alias (t, s, a) for short reference in OUT{}
- Multiple versions allowed (s, s2), select final in OUT{}
- Natural prose mixed with declarations


## 3. LNDL Action Syntax - Two Patterns

LNDL supports two action patterns:
1. **Namespaced**: `<lact Model.field alias>function(args)</lact>` - for mixing with lvars
2. **Direct**: `<lact name>function(args)</lact>` - entire output from action

In [4]:
# Example: Direct actions (entire output)
example_direct = """
Let me search for relevant data:

<lact broad>search(query="AI", limit=100)</lact>
Too much noise. Let me refine:
<lact focused>search(query="AI safety", limit=20)</lact>

Now analyze:
<lvar Report.title t>AI Safety Analysis</lvar>
<lvar Report.summary s>Based on search results...</lvar>

```lndl
OUT{report:[t, s], search_data:[focused], quality_score:0.85}
```
"""

print("Direct Action Pattern:")
print(example_direct)
print("\nKey Points:")
print("- Only 'focused' action executes (referenced in OUT{})")
print("- 'broad' action is scratch work (not in OUT{}, won't execute)")
print("- Direct actions provide entire field value")

Direct Action Pattern:

Let me search for relevant data:

<lact broad>search(query="AI", limit=100)</lact>
Too much noise. Let me refine:
<lact focused>search(query="AI safety", limit=20)</lact>

Now analyze:
<lvar Report.title t>AI Safety Analysis</lvar>
<lvar Report.summary s>Based on search results...</lvar>

```lndl
OUT{report:[t, s], search_data:[focused], quality_score:0.85}
```


Key Points:
- Only 'focused' action executes (referenced in OUT{})
- 'broad' action is scratch work (not in OUT{}, won't execute)
- Direct actions provide entire field value


In [5]:
# Example: Namespaced actions (mixing with lvars)
example_namespaced = """
Building a report with mixed content:

<lvar Report.title t>Analysis Report</lvar>
<lact Report.summary summarize>generate_summary(data="metrics")</lact>
<lvar Report.footer f>End of Report</lvar>

```lndl
OUT{report:[t, summarize, f]}
```
"""

print("Namespaced Action Pattern:")
print(example_namespaced)
print("\nKey Points:")
print("- Model.field namespacing (Report.summary) allows mixing")
print("- Can combine lvars (t, f) and actions (summarize) in same model")
print("- Action result fills Report.summary field")

Namespaced Action Pattern:

Building a report with mixed content:

<lvar Report.title t>Analysis Report</lvar>
<lact Report.summary summarize>generate_summary(data="metrics")</lact>
<lvar Report.footer f>End of Report</lvar>

```lndl
OUT{report:[t, summarize, f]}
```


Key Points:
- Model.field namespacing (Report.summary) allows mixing
- Can combine lvars (t, f) and actions (summarize) in same model
- Action result fills Report.summary field


## 4. Output Syntax - OUT{} Block

The OUT{} block specifies the final structured output using declared variables and actions.

In [6]:
# Example output patterns
output_examples = {
    "Model with lvars": "OUT{report:[title, summary, author]}",
    "Model with mixed": "OUT{report:[title, api_call, summary]}",
    "Model from action": "OUT{data:[fetch_data]}",
    "Scalar literal": "OUT{quality_score:0.85}",
    "Scalar from var": "OUT{quality_score:[q]}",
    "Scalar from action": "OUT{score:[calculate]}",
    "Multiple fields": "OUT{report:[t, s], score:0.9, is_draft:false}",
}

print("OUT{} Block Patterns:\n")
for pattern, example in output_examples.items():
    print(f"{pattern:20s} → {example}")

print("\nRules:")
print("- Models: Use array syntax field:[var1, var2]")
print("- Scalars: Direct literals (0.8, false) or single var/action")
print("- Actions: Referenced by alias, only execute if in OUT{}")
print("- Mixing: Combine lvars and namespaced actions in arrays")

OUT{} Block Patterns:

Model with lvars     → OUT{report:[title, summary, author]}
Model with mixed     → OUT{report:[title, api_call, summary]}
Model from action    → OUT{data:[fetch_data]}
Scalar literal       → OUT{quality_score:0.85}
Scalar from var      → OUT{quality_score:[q]}
Scalar from action   → OUT{score:[calculate]}
Multiple fields      → OUT{report:[t, s], score:0.9, is_draft:false}

Rules:
- Models: Use array syntax field:[var1, var2]
- Scalars: Direct literals (0.8, false) or single var/action
- Actions: Referenced by alias, only execute if in OUT{}
- Mixing: Combine lvars and namespaced actions in arrays


## 5. Natural Thinking Integration

LNDL allows natural prose, reasoning, and revisions alongside structured declarations.

In [7]:
# Example: Natural thinking with LNDL
natural_thinking = """
Let me analyze this step by step.

First, I'll search for background information:
<lact background>search(query="AI ethics", limit=10)</lact>

Hmm, that might not be enough. Let me also search for recent developments:
<lact recent>search(query="AI ethics 2025", limit=5)</lact>

Now I'll draft the title:
<lvar Report.title t1>Ethics in AI</lvar>

Actually, let me be more specific:
<lvar Report.title t2>Ethical Considerations in Modern AI Systems</lvar>

Much better. For the summary, I'll use the recent search:
<lvar Report.summary s>Based on 2025 developments...</lvar>

And add confidence:
<lvar Report.confidence c>0.85</lvar>

Final output uses refined title (t2) and recent search:
```lndl
OUT{report:[t2, s], search_results:[recent], confidence:[c]}
```
"""

print("Natural Thinking Example:")
print(natural_thinking)
print("\nKey Features:")
print("- Prose reasoning intermixed with declarations")
print("- Multiple versions explored (t1, t2)")
print("- Unused elements ignored (background, t1)")
print("- Final selection in OUT{} block")

Natural Thinking Example:

Let me analyze this step by step.

First, I'll search for background information:
<lact background>search(query="AI ethics", limit=10)</lact>

Hmm, that might not be enough. Let me also search for recent developments:
<lact recent>search(query="AI ethics 2025", limit=5)</lact>

Now I'll draft the title:
<lvar Report.title t1>Ethics in AI</lvar>

Actually, let me be more specific:
<lvar Report.title t2>Ethical Considerations in Modern AI Systems</lvar>

Much better. For the summary, I'll use the recent search:
<lvar Report.summary s>Based on 2025 developments...</lvar>

And add confidence:
<lvar Report.confidence c>0.85</lvar>

Final output uses refined title (t2) and recent search:
```lndl
OUT{report:[t2, s], search_results:[recent], confidence:[c]}
```


Key Features:
- Prose reasoning intermixed with declarations
- Multiple versions explored (t1, t2)
- Unused elements ignored (background, t1)
- Final selection in OUT{} block


## 6. Common Errors and Corrections

LNDL syntax has specific patterns to avoid common mistakes.

In [8]:
# Common errors with corrections
errors = {
    "Missing Model.field prefix": {
        "wrong": "<lvar title>value</lvar>",
        "right": "<lvar Report.title t>value</lvar>",
        "reason": "Must specify Model.field for explicit mapping",
    },
    "Mismatched tags": {
        "wrong": "<lvar Report.title t>value</var>",
        "right": "<lvar Report.title t>value</lvar>",
        "reason": "Opening and closing tags must match",
    },
    "Constructor syntax": {
        "wrong": "OUT{report:Report(title=t, summary=s)}",
        "right": "OUT{report:[t, s]}",
        "reason": "Use array syntax, not constructors",
    },
    "Scalar array syntax": {
        "wrong": "OUT{quality_score:[x, y]}",
        "right": "OUT{quality_score:[x]} or OUT{quality_score:0.85}",
        "reason": "Scalars need single var/action or literal",
    },
    "Name collision": {
        "wrong": "<lact Report.field data>...</lact>\n<lvar Report.field data>...</lvar>\nOUT{field:[data]}",
        "right": "<lact Report.field action_data>...</lact>\n<lvar Report.field var_data>...</lvar>\nOUT{field:[action_data]}",
        "reason": "Aliases must be unique (can't have both lvar and lact with same name)",
    },
}

print("Common LNDL Errors:\n")
for error_type, details in errors.items():
    print(f"❌ {error_type}:")
    print(f"   Wrong: {details['wrong']}")
    print(f"   Right: {details['right']}")
    print(f"   Why:   {details['reason']}")
    print()

Common LNDL Errors:

❌ Missing Model.field prefix:
   Wrong: <lvar title>value</lvar>
   Right: <lvar Report.title t>value</lvar>
   Why:   Must specify Model.field for explicit mapping

❌ Mismatched tags:
   Wrong: <lvar Report.title t>value</var>
   Right: <lvar Report.title t>value</lvar>
   Why:   Opening and closing tags must match

❌ Constructor syntax:
   Wrong: OUT{report:Report(title=t, summary=s)}
   Right: OUT{report:[t, s]}
   Why:   Use array syntax, not constructors

❌ Scalar array syntax:
   Wrong: OUT{quality_score:[x, y]}
   Right: OUT{quality_score:[x]} or OUT{quality_score:0.85}
   Why:   Scalars need single var/action or literal

❌ Name collision:
   Wrong: <lact Report.field data>...</lact>
<lvar Report.field data>...</lvar>
OUT{field:[data]}
   Right: <lact Report.field action_data>...</lact>
<lvar Report.field var_data>...</lvar>
OUT{field:[action_data]}
   Why:   Aliases must be unique (can't have both lvar and lact with same name)



## 7. Lazy Execution - Actions Only Run When Referenced

A key feature: actions are only executed if they appear in the OUT{} block.

In [9]:
# Example demonstrating lazy execution
lazy_execution_example = """
Let me explore different search strategies:

Strategy 1 - Broad search:
<lact strategy1>search(query="AI", limit=1000)</lact>
This would be expensive and noisy.

Strategy 2 - Focused search:
<lact strategy2>search(query="AI safety research 2025", limit=20)</lact>
This is more targeted.

Strategy 3 - Academic only:
<lact strategy3>search(query="AI safety", domain=".edu", limit=15)</lact>
This gives authoritative sources.

I'll go with strategy 2:
```lndl
OUT{search_results:[strategy2]}
```
"""

print("Lazy Execution Example:")
print(lazy_execution_example)
print("\nExecution Behavior:")
print("✓ strategy2: EXECUTES (in OUT{})")
print("✗ strategy1: NOT executed (scratch work)")
print("✗ strategy3: NOT executed (scratch work)")
print("\nBenefit: Can explore multiple approaches without cost")

Lazy Execution Example:

Let me explore different search strategies:

Strategy 1 - Broad search:
<lact strategy1>search(query="AI", limit=1000)</lact>
This would be expensive and noisy.

Strategy 2 - Focused search:
<lact strategy2>search(query="AI safety research 2025", limit=20)</lact>
This is more targeted.

Strategy 3 - Academic only:
<lact strategy3>search(query="AI safety", domain=".edu", limit=15)</lact>
This gives authoritative sources.

I'll go with strategy 2:
```lndl
OUT{search_results:[strategy2]}
```


Execution Behavior:
✓ strategy2: EXECUTES (in OUT{})
✗ strategy1: NOT executed (scratch work)
✗ strategy3: NOT executed (scratch work)

Benefit: Can explore multiple approaches without cost


## 8. Type Categories - Scalars vs Models vs Actions

LNDL distinguishes between three output types with different syntax requirements.

In [10]:
# Type-specific examples
type_examples = """
=== SCALARS (float, str, int, bool) ===

Direct literal:
OUT{quality:0.8, is_draft:false, count:42, status:"pending"}

From variable:
<lvar Metrics.quality q>0.85</lvar>
OUT{quality:[q]}

From action:
<lact score>calculate_score(data="metrics")</lact>
OUT{quality:[score]}

=== MODELS (Pydantic classes) ===

From lvars:
<lvar Report.title t>Title</lvar>
<lvar Report.summary s>Summary</lvar>
OUT{report:[t, s]}

Mixed lvars and namespaced actions:
<lvar Report.title t>Title</lvar>
<lact Report.summary gen>generate_summary()</lact>
<lvar Report.footer f>Footer</lvar>
OUT{report:[t, gen, f]}

From direct action (entire model):
<lact fetch>fetch_report(id=123)</lact>
OUT{report:[fetch]}

=== ACTIONS (tool/function calls) ===

Namespaced (for mixing):
<lact Model.field name>function(args)</lact>

Direct (entire output):
<lact name>function(args)</lact>

Pythonic syntax:
<lact data>search(query="AI", limit=10, sort="relevance")</lact>
"""

print(type_examples)


=== SCALARS (float, str, int, bool) ===

Direct literal:
OUT{quality:0.8, is_draft:false, count:42, status:"pending"}

From variable:
<lvar Metrics.quality q>0.85</lvar>
OUT{quality:[q]}

From action:
<lact score>calculate_score(data="metrics")</lact>
OUT{quality:[score]}

=== MODELS (Pydantic classes) ===

From lvars:
<lvar Report.title t>Title</lvar>
<lvar Report.summary s>Summary</lvar>
OUT{report:[t, s]}

Mixed lvars and namespaced actions:
<lvar Report.title t>Title</lvar>
<lact Report.summary gen>generate_summary()</lact>
<lvar Report.footer f>Footer</lvar>
OUT{report:[t, gen, f]}

From direct action (entire model):
<lact fetch>fetch_report(id=123)</lact>
OUT{report:[fetch]}

=== ACTIONS (tool/function calls) ===

Namespaced (for mixing):
<lact Model.field name>function(args)</lact>

Direct (entire output):
<lact name>function(args)</lact>

Pythonic syntax:
<lact data>search(query="AI", limit=10, sort="relevance")</lact>



## 9. Practical Usage - Integrating with LLM

How to use the LNDL prompt in actual LLM interactions.

In [11]:
# Example integration pattern
def create_lndl_instruction(task_description: str, specs: str) -> dict:
    """Create a complete LLM instruction with LNDL guidance.

    Args:
        task_description: What the LLM should accomplish
        specs: Model/field specifications

    Returns:
        Dict with system and user messages
    """
    system_message = get_lndl_system_prompt()

    user_message = f"""
Task: {task_description}

Specifications:
{specs}

Please provide your response using LNDL syntax.
"""

    return {"system": system_message, "user": user_message}


# Example usage
instruction = create_lndl_instruction(
    task_description="Analyze AI safety research and create a report",
    specs="""
    - report(Report: title, summary, conclusion)
    - search_results(SearchData: items, count)
    - confidence_score(float)
    """,
)

print("System Message (first 300 chars):")
print(instruction["system"][:300])
print("...\n")
print("User Message:")
print(instruction["user"])

System Message (first 300 chars):
LNDL - Structured Output with Natural Thinking

SYNTAX

Variables:
<lvar Model.field alias>value</lvar>

- Model.field: Explicit mapping (Report.title, Reason.confidence)
- alias: Short name for OUT{} reference (optional, defaults to field name)
- Declare anywhere, revise anytime, think naturally

A
...

User Message:

Task: Analyze AI safety research and create a report

Specifications:

    - report(Report: title, summary, conclusion)
    - search_results(SearchData: items, count)
    - confidence_score(float)
    

Please provide your response using LNDL syntax.



## 10. Complete Example - End-to-End Workflow

A full example showing LNDL in a realistic scenario.

In [12]:
# Complete LNDL response example
complete_example = """
I need to research AI safety and create a comprehensive report.

First, let me gather data from multiple sources:

<lact academic>search(query="AI safety research", domain=".edu", limit=15)</lact>

Good academic sources. Now recent industry news:
<lact industry>search(query="AI safety 2025", limit=10)</lact>

And get some metrics:
<lact metrics>get_safety_metrics(timeframe="2025")</lact>

Now I'll draft the report:

<lvar Report.title t1>AI Safety in 2025</lvar>

Let me be more descriptive:
<lvar Report.title t2>Comprehensive Analysis of AI Safety Practices in 2025</lvar>

For the summary, I'll combine insights:
<lvar Report.summary s>This analysis examines current AI safety practices based on academic research and industry developments. Key findings show...</lvar>

For conclusion, I'll use the metrics data:
<lact Report.conclusion conclusion_gen>generate_conclusion(metrics_data=[metrics])</lact>

Calculate confidence based on source quality:
<lact conf>calculate_confidence(sources=[academic, industry])</lact>

Final output - using refined title (t2), manually written summary (s),
generated conclusion (conclusion_gen), and both search results:

```lndl
OUT{
  report:[t2, s, conclusion_gen],
  academic_sources:[academic],
  industry_sources:[industry],
  confidence:[conf]
}
```
"""

print("Complete LNDL Example:")
print(complete_example)
print("\nExecution Summary:")
print("EXECUTED actions (in OUT{}):")
print("  - academic: search academic sources")
print("  - industry: search industry news")
print("  - conclusion_gen: generate conclusion")
print("  - conf: calculate confidence")
print("\nNOT EXECUTED (scratch work):")
print("  - metrics: get_safety_metrics (not in OUT{})")
print("\nVariables used:")
print("  - t2: refined title (t1 discarded)")
print("  - s: summary")
print("  - conclusion_gen: generated conclusion")

Complete LNDL Example:

I need to research AI safety and create a comprehensive report.

First, let me gather data from multiple sources:

<lact academic>search(query="AI safety research", domain=".edu", limit=15)</lact>

Good academic sources. Now recent industry news:
<lact industry>search(query="AI safety 2025", limit=10)</lact>

And get some metrics:
<lact metrics>get_safety_metrics(timeframe="2025")</lact>

Now I'll draft the report:

<lvar Report.title t1>AI Safety in 2025</lvar>

Let me be more descriptive:
<lvar Report.title t2>Comprehensive Analysis of AI Safety Practices in 2025</lvar>

For the summary, I'll combine insights:
<lvar Report.summary s>This analysis examines current AI safety practices based on academic research and industry developments. Key findings show...</lvar>

For conclusion, I'll use the metrics data:
<lact Report.conclusion conclusion_gen>generate_conclusion(metrics_data=[metrics])</lact>

Calculate confidence based on source quality:
<lact conf>calculat

## Summary Checklist

**LNDL Prompt Essentials:**
- ✅ System prompt via `get_lndl_system_prompt()`
- ✅ Variables: `<lvar Model.field alias>value</lvar>`
- ✅ Namespaced actions: `<lact Model.field alias>func()</lact>`
- ✅ Direct actions: `<lact name>func()</lact>`
- ✅ Output block: `OUT{field:[vars], scalar:literal}`
- ✅ Lazy execution: Only referenced actions execute
- ✅ Natural thinking: Prose + declarations intermixed
- ✅ Type safety: Model.field namespacing prevents ambiguity

**Key Rules:**
- Models use array syntax: `field:[var1, var2]`
- Scalars use literals or single var/action: `field:0.8` or `field:[x]`
- Actions only execute if in OUT{} block
- Aliases must be unique (no lvar/lact name collision)
- Multiple versions allowed, select final in OUT{}

**Next Steps:**
- See `Parser` for parsing LNDL-formatted LLM responses
- See `Validator` for validating LNDL syntax
- See lionherd operations for integration examples